In [16]:
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
import requests
import re
import pandas as pd

In [17]:
load_dotenv()

API_KEY = os.getenv("PERPLEXITY_API_KEY")
if not API_KEY:
    raise ValueError("Missing or empty PERPLEXITY_API_KEY environment variable")

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "accept": "application/json",
}

In [18]:
def perplexity_query(messages):
    url = "https://api.perplexity.ai/chat/completions"
    json_data = {
        "model": "sonar-reasoning",
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 500,
    }
    response = requests.post(url, headers=headers, json=json_data, timeout=30)
    response.raise_for_status()
    data = response.json()
    return data["choices"][0]["message"]["content"]

In [19]:
df = pd.read_csv("sensor_data.csv", parse_dates=["timestamp"])

In [20]:
print("Preview of 5 entries in sensor_data.csv:")
print(df.head())

Preview of 5 entries in sensor_data.csv:
   id  soil_moisture  temperature  wind_speed  wind_direction  light  \
0   1          26.28         15.2        11.8             146    372   
1   2          26.68         16.4         9.1             149    557   
2   3          36.44         30.1        12.8             351    562   
3   4          34.47         32.7         2.9              11    524   
4   5          29.97         27.7         1.1             171    686   

   uv_index  humidity  pressure  leaf_moisture           timestamp  
0       9.1        33      1007             35 2025-11-03 02:02:46  
1       3.3        72       996             26 2025-11-03 02:32:46  
2       5.1        66      1013             11 2025-11-03 03:02:46  
3       7.2        32      1016             32 2025-11-03 03:32:46  
4       8.4        48       997             28 2025-11-03 04:02:46  


In [21]:
class State(TypedDict):
    question: str
    data_summary: str  # Replace query with summarized data string
    result: str
    answer: str

In [22]:
def generate_data_summary(state: State) -> State:
    recent = df[df["timestamp"] >= pd.Timestamp.now() - pd.Timedelta(days=1)]
    soil_moisture_avg = recent["soil_moisture"].mean()
    temp_avg = recent["temperature"].mean()
    humidity_avg = recent["humidity"].mean()
    summary = (
        f"Recent soil moisture average: {soil_moisture_avg:.2f}, "
        f"temperature average: {temp_avg:.1f}C, "
        f"humidity average: {humidity_avg:.1f}%."
    )
    state["data_summary"] = summary
    return state

In [23]:
def run_dummy_query(state: State) -> State:
    state["result"] = state.get("data_summary", "")
    return state

In [24]:
def generate_decision(state: State) -> State:
    messages = [
        {"role": "system", "content": "Based on sensor data summary and question, provide an agricultural decision or answer."},
        {"role": "user", "content": f"Sensor data summary: {state['data_summary']}\nQuestion: {state['question']}"}
    ]
    answer = perplexity_query(messages)
    state["answer"] = answer
    return state

In [25]:
graph = StateGraph(State)
graph.add_node("generate_data_summary", generate_data_summary)
graph.add_node("dummy_query", run_dummy_query)
graph.add_node("generate_decision", generate_decision)
graph.add_edge(START, "generate_data_summary")
graph.add_edge("generate_data_summary", "dummy_query")
graph.add_edge("dummy_query", "generate_decision")
graph.add_edge("generate_decision", END)

In [26]:
agent = graph.compile()

In [27]:
initial_state = {
    "question": "Should I irrigate my crop field today based on current soil moisture and weather?",
    "data_summary": "",
    "result": "",
    "answer": "",
}

In [28]:
result_state = agent.invoke(initial_state)

In [29]:
print(f"Final decision/answer: {result_state['answer']}")

Final decision/answer: <think>
Let me analyze the provided information to determine whether irrigation is needed for the crop field today.

First, let's understand the sensor data:
- Soil moisture average: 24.13% (volumetric water content)
- Temperature average: 26.0°C
- Humidity average: 53.5%

I need to determine if this soil moisture level is sufficient for the crop or if irrigation is needed. However, there are some important factors missing:
1. What crop is being grown?
2. What is the soil type?
3. What is the crop's growth stage?
4. What are the specific thresholds for this crop and soil type?

Let me review the search results to understand how to make this irrigation decision.

From search result [1], I see that irrigation decisions should be based on:
- Soil moisture content
- Management Allowable Depletion (MAD) levels
- Field capacity and permanent wilting point values for the specific soil type
- Rooting depth of the crop

From search result [3], I learn that irrigation invo